# 1. Introduction to this notebook

The purpose of this notebook is to explain the process followed to load the scraped (extracted) and transformed data in 2 ways:
- Saved as checkpoint .csv files in the local machine
- Loaded as table records into a databe.

# 2. Save as csv

The process of loading the files as csv are quite straight forward and is performed via the ``save_csv()`` function, that can be found at `src/support/data_load_support.py`.

# 3. Load to database

The process to load as a database includes:
- Creating a database
- Creating the tables where the records will be inserted
- Inserting the transformed records into the database

For that matter, a database structure design must be created, for which the following is the proposed SQL schema:

```sql
DROP TABLE IF EXISTS products, categories, subcategories, prices, supermarkets, supermarket_product, brands CASCADE;

CREATE TABLE categories (
    category_id SERIAL PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
);

CREATE TABLE subcategories (
    subcategory_id SERIAL PRIMARY KEY,
    category_id INT REFERENCES categories(category_id) ON DELETE CASCADE ON UPDATE CASCADE, -- if category is changed, change here too. If deleted, subcategory no longer needed
    subcategory_name VARCHAR(100) NOT NULL,
    distinction VARCHAR(100),
    eco BOOLEAN
);

CREATE TABLE brands (
    brand_id SERIAL PRIMARY KEY,
    brand_name VARCHAR(100) NOT NULL
);

CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    brand_id INT REFERENCES brands(brand_id) ON DELETE SET NULL ON UPDATE CASCADE, -- if brand_id is changed, change here too. if deleted, remove category from product
    subcategory_id INT REFERENCES subcategories(subcategory_id) ON DELETE SET NULL ON UPDATE CASCADE, -- if category is changed, change here too. if deleted, remove category from product
    product_name_norm VARCHAR(200) NOT NULL,
    quantity NUMERIC,
    units VARCHAR(50),
    volume_weight NUMERIC
);

CREATE TABLE supermarkets (
    supermarket_id SERIAL PRIMARY KEY,
    supermarket_name VARCHAR(100) NOT NULL,
    product_name_supermarket VARCHAR(200)
);

CREATE TABLE supermarket_product (
    supermarket_product_id SERIAL PRIMARY KEY,
    supermarket_id INT REFERENCES supermarkets(supermarket_id) ON DELETE CASCADE ON UPDATE CASCADE, -- if supermarket is deleted, interation is no longer needed
    product_id INT REFERENCES products(product_id) ON DELETE CASCADE ON UPDATE CASCADE, -- if product is deleted, interation is no longer needed
    facua_url VARCHAR(255)
);

CREATE TABLE prices (
    price_id SERIAL PRIMARY KEY,
    supermarket_product_id INT REFERENCES supermarket_product(supermarket_product_id) ON DELETE CASCADE ON UPDATE CASCADE, -- if category is changed, change here too. If deleted, subcategory no longer needed
    date DATE NOT NULL,
    price_amount NUMERIC(10, 2) NOT NULL
);
```

Each table has been thought as to keep the minimum double information as possible (normalization). For that reason, the intermediate table between products and supermarkets has been made, where products is a product standardization table, supermarkets is self explanatory and the intermediate is where the whole denomination of each given product at each given supermarket is provided. From that supermarket_product table id, the historical prices relation is obtained.

To the normalization performed, there is perhaps the exception of "subcategories", where another extra table could have been created for distinctions and eco tag. But given the negligible size memory of this project and the already substantial amount of JOINs that it requires to gather all the information, it was considered non-vital, albeit recognizably less efficient.


## 3.1 Database and record insertion approach


For this data schema to be created, a database must have been created previously. This is has been done through DBeaver's GUI rather than through code, for this project, due to its ease. The creation of the tables, however, has been done through code and the tables are dropped and re-created each time the ETL runs.

The dropping and recreation of the tables is like so because of the exploratory nature of this project. It ensures a clean run every time. However, if the project were to be extended, registering the history of prices for a longer period, then it would make sense to modify the ETL to only ingest the latest price and then not drop the tables everytime, just insert a new record per product.

## 3.2 Database schema

To consult how the tables are created and how records are inserted, please refer to `src/support/data_load_support.py`. The database Entity-Relation diagram once the tables are created is the following:

![E-R-diagram](../assets/entity_relation_diagram.png)

## 3.3 Data insertion order

The trickiest part of the data load pipeline is correctly setting the order of insertion for the tables at hand. It is necessary to start by the tables that bear no reference to the others. Given that constraint, the order of loading as per the insert functions at `src/support/data_load_support.py` integrated in `src/data_etl.py` is:

1. Categories
2. Subcategories 
3. Brands
4. Supermarkets
5. Products
6. Supermarkets_products
7. Prices

Each of the insert functions at `src/support/data_load_support.py` performs a SELECT to check if the record at hand already exists. If it does not, it is created. Either way, the id of the product is returned by the function so the next referenced table insert has it as a Foreign Key.

# 4. Conclusion of this notebook

If you wish to consult the analysis extracted ouf ot this data, please refer to the notebook at `notebooks/4_data_analysis.ipynb`.